In [25]:
import pandas as pd
import numpy as np
from tqdm import tqdm 
pd.set_option('display.max_columns', 500)
path = '../data/weather/'

In [2]:
city = pd.read_csv('../data/weather/city_attributes.csv')

In [3]:
city.head()

,City,Country,Latitude,Longitude
0,Vancouver,Canada,49.249660,-123.119339
1,Portland,United States,45.523449,-122.676208
2,San Francisco,United States,37.774929,-122.419418
3,Seattle,United States,47.606209,-122.332069
4,Los Angeles,United States,34.052231,-118.243683


In [4]:
city['City'].nunique()

36

In [5]:
train = pd.read_csv('../data/other/sales_train_validation.csv.zip')

In [130]:
# train['store_id'].unique()

In [7]:
# TX:テキサス
# CA:カルフォルニア
# WI:ウィスコンシン

In [8]:
TX = ['San Antonio','Dallas','Houston']
CA = ['San Francisco','Los Angeles','San Diego']
WI = []

In [9]:
choice_city = city[city['City'].isin(TX+CA)].reset_index(drop=True)

In [32]:
def create_df(dataset,name=None):
    use_col = ['datetime']+TX+CA
    c_dataset =  dataset[use_col][dataset['datetime'] >= '2012-10-02'].reset_index(drop=True)
    c_dataset['Date'] = [i[:10] for i in c_dataset['datetime']]

    df = pd.DataFrame()
    list_region = []
    list_High = []
    list_Low = []
    list_Mean = []
    list_date = []

    for (date,data) in tqdm(c_dataset.groupby('Date')):
        for region in TX+CA:
            list_region.append(region)
            list_High.append(data[region].max())
            list_Low.append(data[region].min())
            list_Mean.append(data[region].mean())
            list_date.append(date)

    df['region'] = list_region
    df[f'highest_{name}'] = list_High
    df[f'lowest_{name}'] = list_Low
    df[f'mean_{name}'] = list_Mean
    df['Date'] = list_date
    return df

## 湿度

In [10]:
humidity = pd.read_csv(path+'humidity.csv')

In [35]:
df_humidity = create_df(humidity,'humidity')

100%|██████████| 1886/1886 [00:02<00:00, 912.54it/s]


In [36]:
df_humidity.head()

,region,highest_humidity,lowest_humidity,mean_humidity,Date
0,San Antonio,90.0,56.0,82.416667,2012-10-02
1,Dallas,84.0,26.0,68.208333,2012-10-02
2,Houston,79.0,34.0,60.000000,2012-10-02
3,San Francisco,94.0,11.0,65.958333,2012-10-02
4,Los Angeles,88.0,12.0,64.916667,2012-10-02


## 気圧

In [17]:
pressure = pd.read_csv(path+'pressure.csv')

In [33]:
df_pressure = create_df(pressure,'pressure')

100%|██████████| 1886/1886 [00:01<00:00, 967.84it/s]


In [37]:
df_pressure.head()

,region,highest_pressure,lowest_pressure,mean_pressure,Date
0,San Antonio,1014.0,1012.0,1013.750000,2012-10-02
1,Dallas,1012.0,1009.0,1010.500000,2012-10-02
2,Houston,1011.0,1007.0,1008.291667,2012-10-02
3,San Francisco,1016.0,1000.0,1011.250000,2012-10-02
4,Los Angeles,1014.0,1009.0,1012.750000,2012-10-02


## 気温

In [40]:
temperature = pd.read_csv(path+'temperature.csv')

In [42]:
df_temperature = create_df(temperature,'temperature')

100%|██████████| 1886/1886 [00:02<00:00, 909.94it/s]


In [44]:
df_temperature.head()

,region,highest_temperature,lowest_temperature,mean_temperature,Date
0,San Antonio,297.16,289.652128,291.860457,2012-10-02
1,Dallas,301.13,290.440902,293.922157,2012-10-02
2,Houston,300.25,288.665253,292.385391,2012-10-02
3,San Francisco,304.86,289.130000,292.958306,2012-10-02
4,Los Angeles,307.71,291.740000,295.890450,2012-10-02


## 天気

In [45]:
weather = pd.read_csv(path+'weather_description.csv')

In [47]:
weather['Vancouver'].unique()

array([nan, 'mist', 'broken clouds', 'sky is clear', 'light rain',
       'few clouds', 'fog', 'overcast clouds',
       'light intensity shower rain', 'moderate rain',
       'light intensity drizzle', 'scattered clouds',
       'proximity shower rain', 'heavy intensity rain', 'heavy snow',
       'shower rain', 'snow', 'heavy shower snow',
       'light intensity drizzle rain', 'light snow', 'very heavy rain',
       'smoke', 'thunderstorm with heavy rain', 'light shower snow',
       'thunderstorm', 'thunderstorm with light rain', 'haze', 'dust',
       'volcanic ash', 'heavy intensity shower rain',
       'thunderstorm with rain', 'sleet', 'light rain and snow',
       'drizzle', 'shower snow', 'light shower sleet',
       'proximity thunderstorm', 'ragged thunderstorm'], dtype=object)

In [69]:
sunny = ['sky is clear']
cloudy = ['mist','broken clouds','few clouds','overcast clouds','scattered clouds','smoke','haze','dust','volcanic ash','sand']
rain = ['light rain','fog','light intensity shower rain','moderate rain','light intensity drizzle','proximity shower rain',\
       'heavy intensity rain','shower rain','light intensity drizzle rain','very heavy rain','thunderstorm with heavy rain',\
       'thunderstorm','thunderstorm with light rain','heavy intensity shower rain','thunderstorm with rain','sleet','drizzle',\
       'light shower sleet','proximity thunderstorm','ragged thunderstorm','squalls','heavy intensity drizzle',\
        'proximity thunderstorm with drizzle','proximity thunderstorm with rain','freezing rain','thunderstorm with light drizzle',\
       'tornado','thunderstorm with heavy drizzle','proximity moderate rain','thunderstorm with drizzle']
snow = ['heavy snow','snow','heavy shower snow','light snow','light shower snow','light rain and snow','shower snow']

In [70]:
weather = weather.replace(sunny,'sunny')
weather = weather.replace(cloudy,'cloudy')
weather = weather.replace(rain,'rain')
weather = weather.replace(snow,'snow')

In [71]:
use_col = ['datetime'] + TX + CA
weather[use_col]

,datetime,San Antonio,Dallas,Houston,San Francisco,Los Angeles,San Diego
0,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-10-01 13:00:00,sunny,cloudy,sunny,rain,cloudy,sunny
2,2012-10-01 14:00:00,sunny,sunny,sunny,sunny,sunny,sunny
3,2012-10-01 15:00:00,sunny,sunny,sunny,sunny,sunny,sunny
4,2012-10-01 16:00:00,sunny,sunny,sunny,sunny,sunny,sunny
...,...,...,...,...,...,...,...
45248,2017-11-29 20:00:00,sunny,cloudy,cloudy,NaN,sunny,cloudy
45249,2017-11-29 21:00:00,sunny,cloudy,cloudy,NaN,sunny,cloudy
45250,2017-11-29 22:00:00,sunny,cloudy,sunny,NaN,sunny,cloudy
45251,2017-11-29 23:00:00,sunny,cloudy,cloudy,NaN,sunny,cloudy


In [73]:
c_weather =  weather[use_col][weather['datetime'] >= '2012-10-02'].reset_index(drop=True)
c_weather['Date'] = [i[:10] for i in c_weather['datetime']]

In [83]:
c_weather['datetime'] = pd.to_datetime(c_weather['datetime'])

In [89]:
c_weather['Hour'] = [Date.hour for Date in c_weather['datetime']]

In [113]:
AM_weather = []
PM_weather = []
list_date = []
list_region = []

df_weather = pd.DataFrame()
for (date,data) in c_weather.groupby('Date'):
    for region in TX+CA:
        list_region.append(region)
        list_date.append(date)
        try:
            AM_weather.append(data[data['Hour'] <=12][region].value_counts().index[0])
        except:
            AM_weather.append(np.nan)
        try:
            PM_weather.append(data[data['Hour'] > 12][region].value_counts().index[0])
        except:
            PM_weather.append(np.nan)

df_weather['region'] = list_region
df_weather['AM_weather'] = AM_weather
df_weather['PM_weather'] = PM_weather
df_weather['Date'] = list_date

In [114]:
df_weather

,region,AM_weather,PM_weather,Date
0,San Antonio,sunny,sunny,2012-10-02
1,Dallas,cloudy,sunny,2012-10-02
2,Houston,cloudy,cloudy,2012-10-02
3,San Francisco,sunny,cloudy,2012-10-02
4,Los Angeles,sunny,sunny,2012-10-02
...,...,...,...,...
11311,Dallas,cloudy,NaN,2017-11-30
11312,Houston,cloudy,NaN,2017-11-30
11313,San Francisco,NaN,NaN,2017-11-30
11314,Los Angeles,sunny,NaN,2017-11-30


In [116]:
len(df_pressure),len(df_temperature),len(df_humidity),len(df_weather)

(11316, 11316, 11316, 11316)

In [117]:
df_temperature

,region,highest_temperature,lowest_temperature,mean_temperature,Date
0,San Antonio,297.16,289.652128,291.860457,2012-10-02
1,Dallas,301.13,290.440902,293.922157,2012-10-02
2,Houston,300.25,288.665253,292.385391,2012-10-02
3,San Francisco,304.86,289.130000,292.958306,2012-10-02
4,Los Angeles,307.71,291.740000,295.890450,2012-10-02
...,...,...,...,...,...
11311,Dallas,287.88,287.880000,287.880000,2017-11-30
11312,Houston,291.44,291.440000,291.440000,2017-11-30
11313,San Francisco,NaN,NaN,NaN,2017-11-30
11314,Los Angeles,294.93,294.930000,294.930000,2017-11-30


In [123]:
drop_col = ['Date','region']
all_weather = pd.concat([df_temperature,df_pressure.drop(drop_col,axis=1),df_humidity.drop(drop_col,axis=1),df_weather.drop(drop_col,axis=1)],axis=1)

In [155]:
sate_weather = pd.DataFrame()
highest_temperature = []
lowest_temperature = []
mean_temperature = []

highest_pressure = []
lowest_pressure = []
mean_pressure = []

highest_humidity = []
lowest_humidity = []
mean_humidity = []

AM_weather = []
PM_weather = []
state = []
list_date = []

for (date,data) in all_weather[all_weather['region'].isin(TX)].groupby('Date'):
    highest_temperature.append(data['highest_temperature'].mean())
    lowest_temperature.append(data['lowest_temperature'].mean())
    mean_temperature.append(data['mean_temperature'].mean())
    
    highest_pressure.append(data['highest_pressure'].mean())
    lowest_pressure.append(data['lowest_pressure'].mean())
    mean_pressure.append(data['mean_pressure'].mean())
    
    highest_humidity.append(data['highest_humidity'].mean())
    lowest_humidity.append(data['lowest_humidity'].mean())
    mean_humidity.append(data['mean_humidity'].mean())
    try:
        AM_weather.append(data['AM_weather'].value_counts().index[0])
    except:
        AM_weather.append(np.nan)
    try:
        PM_weather.append(data['PM_weather'].value_counts().index[0])
    except:
        PM_weather.append(np.nan)
    
    state.append('TX')
    list_date.append(date)
    
for (date,data) in all_weather[all_weather['region'].isin(CA)].groupby('Date'):
    highest_temperature.append(data['highest_temperature'].mean())
    lowest_temperature.append(data['lowest_temperature'].mean())
    mean_temperature.append(data['mean_temperature'].mean())
    
    highest_pressure.append(data['highest_pressure'].mean())
    lowest_pressure.append(data['lowest_pressure'].mean())
    mean_pressure.append(data['mean_pressure'].mean())
    
    highest_humidity.append(data['highest_humidity'].mean())
    lowest_humidity.append(data['lowest_humidity'].mean())
    mean_humidity.append(data['mean_humidity'].mean())
    
    try:
        AM_weather.append(data['AM_weather'].value_counts().index[0])
    except:
        AM_weather.append(np.nan)
    try:
        PM_weather.append(data['PM_weather'].value_counts().index[0])
    except:
        PM_weather.append(np.nan)
    state.append('CA')
    list_date.append(date)
    
sate_weather['state_id'] = state
sate_weather['date'] = list_date

sate_weather['highest_temperature'] = highest_temperature
sate_weather['lowest_temperature'] = lowest_temperature
sate_weather['mean_temperature'] = mean_temperature

sate_weather['highest_pressure'] = highest_pressure
sate_weather['lowest_pressure'] = lowest_pressure
sate_weather['mean_pressure'] = mean_pressure

sate_weather['highest_humidity'] = highest_humidity
sate_weather['lowest_humidity'] = lowest_humidity
sate_weather['mean_humidity'] = mean_humidity

sate_weather['AM_weather'] = AM_weather
sate_weather['PM_weather'] = PM_weather

In [156]:
sate_weather['state_id'] = sate_weather['state_id'].replace({'CA':0,'TX':1})

In [157]:
sate_weather.to_pickle('../data/other/weather.pkl')

In [154]:
sate_weather

,state_id,Date,highest_temperature,lowest_temperature,mean_temperature,highest_pressure,lowest_pressure,mean_pressure,highest_humidity,lowest_humidity,mean_humidity,AM_weather,PM_weather
0,1,2012-10-02,299.513333,289.586094,292.722668,1012.333333,1009.333333,1010.847222,84.333333,38.666667,70.208333,cloudy,sunny
1,1,2012-10-03,299.623333,290.130000,294.124097,1016.333333,1010.333333,1013.361111,83.000000,32.333333,62.152778,cloudy,cloudy
2,1,2012-10-04,301.693333,287.903333,293.889444,1017.666667,1012.666667,1015.333333,97.333333,35.333333,64.083333,cloudy,cloudy
3,1,2012-10-05,303.550000,292.483333,297.188750,1020.333333,1015.333333,1018.000000,86.000000,40.000000,66.444444,cloudy,cloudy
4,1,2012-10-06,302.946667,293.513333,298.111667,1020.000000,1015.333333,1018.444444,93.666667,46.333333,70.208333,cloudy,cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3767,0,2017-11-26,296.115000,284.125000,289.021042,1020.000000,1016.000000,1018.125000,100.000000,55.000000,87.458333,rain,cloudy
3768,0,2017-11-27,293.985000,287.155000,289.195417,1018.000000,1014.500000,1016.270833,96.500000,50.500000,79.333333,cloudy,cloudy
3769,0,2017-11-28,294.410000,281.225000,286.636646,1017.000000,970.500000,1013.187500,87.000000,41.000000,65.666667,sunny,sunny
3770,0,2017-11-29,294.350000,283.995000,288.753125,1017.500000,1014.000000,1015.770833,93.000000,38.500000,66.791667,sunny,sunny


In [138]:
train_pk = pd.read_pickle('../data/not_fe/train_393.pkl')

In [148]:
train_pk[train_pk['id'] == 'HOBBIES_1_001_CA_1_validation']

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,train,2016-04-24,11613,30,4,0,0,0,0,0,8.382812
30490,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,train,2016-04-23,11613,30,4,0,0,0,0,0,8.382812
60980,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2016-04-22,11612,30,4,0,0,0,0,0,8.382812
91470,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,3,train,2016-04-21,11612,30,4,0,0,0,0,0,8.382812
121960,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,train,2016-04-20,11612,30,4,0,0,0,0,0,8.382812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11830120,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2015-04-02,11509,30,4,0,0,1,0,1,8.257812
11860610,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2015-04-01,11509,30,4,0,0,1,1,0,8.257812
11891100,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2015-03-31,11509,30,4,0,0,0,0,0,8.257812
11921590,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,train,2015-03-30,11509,30,4,0,0,0,0,0,8.257812


In [147]:
train[train['state_id'] == 'CA']

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1

In [ ]:
#CA = 0
#TX = 1

In [168]:
ddd = pd.read_pickle('../features/Weather_test.pkl')

In [170]:
X_train = pd.read_pickle('../data/input/train_1y.pkl')

In [171]:
X_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_t28,lag_t29,lag_t30,rolling_mean_t7,rolling_std_t7,rolling_mean_t30,rolling_mean_t90,rolling_mean_t180,rolling_std_t30,rolling_skew_t30,rolling_kurt_t30,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,year,month,week,day,dayofweek
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2015-03-26,11508,30,4,2,2,0,0,0,8.26,2.0,0.0,1.0,0.714286,0.755929,0.666667,0.688889,0.666667,0.994236,1.420145,0.955613,0.0,0.0000,6.000000e-08,0.0,2015,3,13,26,3
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,0,train,2015-03-26,11508,30,4,2,2,0,0,0,3.97,0.0,0.0,0.0,0.000000,0.000000,0.100000,0.300000,0.316667,0.305129,2.809120,6.308054,0.0,0.0000,0.000000e+00,0.0,2015,3,13,26,3
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,0,train,2015-03-26,11508,30,4,2,2,0,0,0,2.97,0.0,0.0,0.0,0.571429,1.133893,0.300000,0.222222,0.161111,0.651259,2.824870,9.577853,0.0,0.0000,0.000000e+00,0.0,2015,3,13,26,3
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,4,train,2015-03-26,11508,30,4,2,2,0,0,0,4.64,1.0,0.0,0.0,2.428571,4.276180,2.366667,2.466667,2.383333,2.976730,2.046232,3.929555,0.0,0.0000,0.000000e+00,0.0,2015,3,13,26,3
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,0,train,2015-03-26,11508,30,4,2,2,0,0,0,2.88,0.0,0.0,0.0,1.000000,1.414214,1.233333,1.211111,1.150000,1.330889,1.420910,1.523538,0.0,0.0647,0.000000e+00,0.0,2015,3,13,26,3


In [178]:
def load_datasets(feats):
    dfs = [pd.read_pickle(f'../features/{f}_train.pkl') for f in feats]
    X_train = pd.concat(dfs, axis=1)
    dfs = [pd.read_pickle(f'../features/{f}_test.pkl') for f in feats]
    X_test = pd.concat(dfs, axis=1)
    return X_train, X_test

In [179]:
feats = ["Base","Date","Price","Time"]

In [180]:
XX_train,XX_test = load_datasets(feats)

In [181]:
XX_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_t28,lag_t29,lag_t30,rolling_mean_t7,rolling_std_t7,rolling_mean_t30,rolling_mean_t90,rolling_mean_t180,rolling_std_t30,rolling_skew_t30,rolling_kurt_t30,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,year,month,week,day,dayofweek


In [176]:
tmp = pd.read_pickle('/Users/zakopuro/Code/python_code/kaggle/kaggle_walmart/temp.pkl')

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,14370,1437,3,1,0,0,1,1,1027,147,-1,-1,-1,-1,0,0,0,598
1,14380,1438,3,1,0,0,0,1,1027,147,-1,-1,-1,-1,0,0,0,305
2,14390,1439,3,1,0,0,1,1,1027,147,-1,-1,-1,-1,0,0,0,230
3,14400,1440,3,1,0,0,2,1,1027,147,-1,-1,-1,-1,0,0,0,353
4,14410,1441,3,1,0,0,4,1,1027,147,-1,-1,-1,-1,0,0,0,223


In [182]:
Base = pd.read_pickle('/Users/zakopuro/Code/python_code/kaggle/kaggle_walmart/features/Base_train.pkl')

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price


In [188]:
tmp['dd'] = 'ddd'

In [190]:
tmp['dd'].dtype == object

True

In [ ]:
import pandas as pd
train = pd.read_pickle('../features/Base_train.pkl')